In [3]:
%pip install langchain langchain-community cohere wikipedia-api pydantic

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import Cohere
from pydantic import BaseModel
from typing import Optional
import wikipediaapi

# Step 1: Set up the Cohere API
COHERE_API_KEY="uOg8fNJyr27I4hfXCxjKGyiD5ZQZWWd8nRhFqbAV"
cohere_llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command")

# Step 2: Download Indian Penal Code (IPC) summary from Wikipedia
def fetch_ipc_summary() -> str:
    wiki = wikipediaapi.Wikipedia(user_agent="IPCChatbot/1.0 (contact: myemail@example.com)", language='en')
    page = wiki.page("Indian Penal Code")
    if not page.exists():
        raise ValueError("❌ The Indian Penal Code page does not exist on Wikipedia.")
    return page.text[:5000]  # Limit to first 5000 characters

ipc_content = fetch_ipc_summary()

# Step 3: Define a Pydantic model for structured responses
class IPCResponse(BaseModel):
    section: Optional[str]
    explanation: Optional[str]

# Step 4: Create a prompt template for the chatbot
prompt_template = PromptTemplate(
    input_variables=["question"],
    template="""
You are a legal assistant chatbot specialized in the Indian Penal Code (IPC).
Refer to the following IPC document content to answer the user's query:

{ipc_content}

User Question: {question}

Provide a detailed answer, mentioning the relevant section if applicable.
"""
)

# Step 5: Function to get IPC response
def get_ipc_response(question: str) -> IPCResponse:
    prompt = prompt_template.format(ipc_content=ipc_content, question=question)
    response = cohere_llm.predict(prompt)

    if "Section" in response:
        section_part = response.split('Section', 1)[1]
        section = section_part.split(':')[0].strip()
        explanation = section_part.split(':', 1)[-1].strip()
    else:
        section = None
        explanation = response.strip()

    return IPCResponse(section=section, explanation=explanation)

# Step 6: Run chatbot in terminal
def display_response(response: IPCResponse):
    print("\n📘 IPC Legal Assistant Response:")
    print(f"📑 Section: {response.section or 'N/A'}")
    print(f"📝 Explanation: {response.explanation}\n")

print("\n👨‍⚖️ Welcome to the Indian Penal Code (IPC) Assistant!")
print("Type your legal question or type 'exit' to quit.\n")

while True:
    question = input("❓ Ask about the Indian Penal Code: ").strip()
    if question.lower() == 'exit':
        print("👋 Exiting IPC Assistant. Goodbye!")
        break
    elif not question:
        print("⚠️ Please enter a valid question.\n")
        continue

    try:
        response = get_ipc_response(question)
        display_response(response)
    except Exception as e:
        print(f"❌ Error: {e}\n")



👨‍⚖️ Welcome to the Indian Penal Code (IPC) Assistant!
Type your legal question or type 'exit' to quit.


📘 IPC Legal Assistant Response:
📑 Section: 302 of the Indian Penal Code addresses the crime of murder. It states that any person convicted of murder shall be punished with imprisonment for life or sentenced to death. 

If you have stumbled upon this section and have details on any offences that might have occurred, it is advised that you consult with a legal professional for a thorough understanding and assistance regarding the proceedings.
📝 Explanation: 302 of the Indian Penal Code addresses the crime of murder. It states that any person convicted of murder shall be punished with imprisonment for life or sentenced to death. 

If you have stumbled upon this section and have details on any offences that might have occurred, it is advised that you consult with a legal professional for a thorough understanding and assistance regarding the proceedings.

⚠️ Please enter a valid questi